<a href="https://colab.research.google.com/github/MoSkibidi/The_Art_Of_Computer_System/blob/main/machine_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Language

*ดัดแปลงจากโน้ตบุ๊คต้นฉบับโดย ผศ.ดร.จิตร์ทัศน์ ฝักเจริญผล ภาคต้น ปีการศึกษา 2563 และอ้างอิงเนื้อหาจากตำรา [The Elements of Computing Systems โดย Nisan และ Schocken](https://www.nand2tetris.org/book)*

บทนี้เราจะศึกษาชุดคำสั่งภาษาเครื่อง (instruction set) ของสถาปัตยกรรม Hack computer นิสิตสามารถอ่านเอกสารประกอบได้ที่ [รากฐานระบบคอมพิวเตอร์: จากเม็ดทรายสู่ Hello, World! บทที่ 4](https://ecourse.cpe.ku.ac.th/courses/comsys/comsys-book.pdf#page=94) เป้าหมายของการทำปฏิบัติการนี้คือ

* ทำความเข้าใจกับสถาปัตยกรรมของ Hack ผ่านทางการเขียนโปรแกรมภาษาแอสเซมบลี้ (Assembly) และดูรหัสภาษาเครื่อง เพื่อเป็นพื้นฐานในการพัฒนา Hack CPU ในบทเรียนหน้า
* สร้างความคุ้นเคยกับกระบวนการแปลภาษาแอสเซมบลี้ไปยังรหัสคำสั่งภาษาเครื่อง เพื่อเตรียมตัวพัฒนาโปรแกรมแอสเซมเบลอร์ (Assembler) ขึ้นมาเองในบทเรียนต่อ ๆ ไป

ก่อนอื่นเราจะโหลดไลบรารีเช่นเคย

In [ ]:
%%capture
!rm -rf comp-sys-public-lib
!wget -q -O - https://ecourse.cpe.ku.ac.th/courses/comsys/lib/ch04.tgz | tar zxf -
!pip install -e comp-sys-public-lib
!pip install -e comp-sys-public-lib/modules/component-builder
import site; site.main()
from course_ch04_init import *

# สถาปัตยกรรม Hack computer

ภายในซีพียูของ Hack computer มีรีจีสเตอร์สองตัวคือ A และ D ตัวละ 16 บิต ซีพียูเชื่อมอยู่กับหน่วยความจำสองชุดที่ใช้แอดเดรสแยกกัน

1. **หน่วยความจำสำหรับเก็บโค้ด (instruction memory)** สร้างขึ้นจาก ROM ขนาด 32K เวิร์ด
2. **หน่วยความจำเก็บข้อมูล (data memory)** ระบุแอดเดรสได้ 32K เวิร์ด แต่ไม่ได้นำมาเก็บข้อมูลได้ทั้งหมด โดยมีการจัดสรรพื้นที่แอดเดรสดังนี้
    * แอดเดรส `0x0000` - `0x3FFF` เป็นหน่วยความจำจริง ๆ ที่ใช้งานได้ สร้างขึ้นจาก RAM ขนาด 16K เวิร์ด
    * แอดเดรส `0x4000` - `0x5FFF` ใช้เข้าถึงหน้าจอแสดงผล
    * แอดเดรส `0x6000` ใช้อ่านค่าจากคีย์บอร์ด

ชุดคำสั่งของ Hack แบ่งเป็นสองกลุ่มคือ

* A-instructions  ใช้สำหรับโหลดค่าเข้ารีจิสเตอร์ A
* C-instructions  สำหรับคำนวณและกระโดด


## A-instruction
A-instruction ใช้สำหรับโหลดค่าคงที่ลงสู่รีจีสเตอร์ A
รูปแบบคำสั่งในรหัสภาษาเครื่อง 16 บิตเป็นดังนี้

              1 1 1 1   1 1
    bit:      5 4 3 2   1 0 9 8   7 6 5 4   3 2 1 0
    A-inst:   0 v v v   v v v v   v v v v   v v v v

โดย `vv...v` แทนค่าคงที่ขนาด 15 บิต

เพื่อให้ง่ายต่อการจดจำและตีความ ภาษาแอสเซมบลี้ของ Hack กำหนดสัญลักษณ์ช่วยจำ หรือ *นีโมนิค (mnemonic)* สำหรับ A-instruction ด้วยอักขระ `@` ดังนี้

    @<value>

โดย `value` ระบุได้ 3 ประเภท ได้แก่

1. ค่าคงที่จำนวนเต็มฐานสิบ
2. *symbol* แทนตำแหน่งในหน่วยความจำ ROM ใช้ระบุตำแหน่งของคำสั่งที่จะกระโดดไปทำงาน (ดูตัวอย่าง `LOOP` ด้านล่าง)
3. *symbol* แทนตำแหน่งในหน่วยความจำ RAM เพื่อใช้แทนตัวแปร รวมถึงคำเฉพาะที่ใช้แทนแอดเดรสพิเศษ เช่น
    * `R0` - `R15` แทนแอดเดรส 0 - 15 (นำมาจำลองเป็นรีจีสเตอร์เสมือน)
    * `SCREEN` แทนแอดเดรสเริ่มต้นของจอแสดงผล (0x4000)
    * `KBD` แทนแอดเดรสของคีย์บอร์ด (0x6000)

  ดูรายการ *symbol* ที่เป็นไปได้ทั้งหมดได้จาก[ตารางที่ 6.1 ของหนังสือ](https://ecourse.cpe.ku.ac.th/courses/comsys/comsys-book.pdf#page=127)

ด้านล่างเป็นตัวอย่างโค้ดภาษาแอสเซมบลี้ที่มีแต่ A-instruction

In [ ]:
ainst_demo = """
    @100
    @200
    @x
(LOOP)
    @50
    @y
    @LOOP
    @R0
    @R1
    @R3
    @SCREEN
"""

ใช้ฟังก์ชัน `listing` ที่ไลบรารีเตรียมไว้ให้เพื่อจัดรูปแบบและแสดงเลขที่อยู่ของคำสั่งในหน่วยความจำรอม ซึ่งมีรูปแบบการใช้งานดังนี้

    listing(asm_code,    # โค้ดภาษาแอสเซมบลี้ในรูปสตริง
            file=None)   # ชื่อไฟล์สำหรับเก็บผลลัพธ์ หากระบุเป็น None จะแสดงผลลัพธ์บนหน้าจอ

ตัวอย่างเช่น

In [ ]:
listing(ainst_demo)

   0: @100
   1: @200
   2: @x
      (LOOP)
   3: @50
   4: @y
   5: @LOOP
   6: @R0
   7: @R1
   8: @R3
   9: @SCREEN


**คำถามชวนคิด:** ให้พิจารณาคำสั่งด้านบน และแยกแยะว่าในแต่ละคำสั่งเป็นการใช้งาน A-instruction ในรูปแบบใด (จากสามแบบที่กล่าวมาข้างต้น)

แม้การตีความคำสั่งภาษาแอสเซมบลี้เป็นรหัสภาษาเครื่องจะค่อนข้างง่ายและตรงไปตรงมา แต่เพื่อความสะดวกในการทำความเข้าใจเนื้อหาส่วนนี้ ไลบรารีของรายวิชาได้เตรียมฟังก์ชัน `assemble` สำหรับแปลโค้ดจากภาษาแอสเซมบลี้เป็นรหัสคำสั่งภาษาเครื่องไว้ให้แล้ว ทดลองใช้งานดังนี้

In [ ]:
insts = assemble(ainst_demo)
print(insts)

ลองนำโค้ดข้างต้นมาพิมพ์ในรูปฐานสอง 16 บิต สังเกตว่าบิตซ้ายสุด (MSB) ของทุกคำสั่งเป็น 0 ทั้งหมด เนื่องจากทุกคำสั่งล้วนเป็น A-instruction

**คำถามชวนคิด:** คำสั่งภาษาเครื่องในบรรทัด `@LOOP` `@x` `@y` และ `@SCREEN`  ถูกสร้างมาได้อย่างไร

In [ ]:
for i in insts:
    print(f'{i:016b}')

เราจะตรวจสอบการทำงานของโค้ดดังกล่าวบนคอมพิวเตอร์จำลองที่ใช้สถาปัตยกรรม Hack ด้วยฟังก์ชัน `simulate` ที่ไลบรารีเตรียมไว้ให้ โดยระบุอาร์กิวเมนต์ `memeory_display_slots` ไปด้วย เพื่อจำกัดตารางหน่วยความจำไม่ให้ยาวเกินไป

In [ ]:
simulate(insts, memory_display_slots=20)

## C-instruction

คำสั่งในกลุ่ม C-instruction ถือเป็นหัวใจหลักของสถาปัตยกรรม Hack คำสั่งในกลุ่มนี้รับผิดชอบเรื่องการคำนวณและตัดสินใจเพื่อควบคุมการไหลของโปรแกรม ภายใน C-instruction หนึ่งคำสั่งมีองค์ประกอบ 3 ส่วนคือ

1. การคำนวณ (computation specification)
2. ที่เก็บผลลัพธ์ (destination specification)
3. รายละเอียดการกระโดด (jump specification)

รูปแบบรหัสภาษาเครื่องของ C-instruction เป็นดังนี้

              1 1 1 1   1 1
    bit:      5 4 3 2   1 0 9 8   7 6 5 4   3 2 1 0
    C-inst:   1 1 1 a   c c c c   c c d d   d j j j

ฟิลด์ `a` และ `cc...c` เรียกรวมกันว่า compute field ใช้กำหนดค่าที่ต้องการคำนวณ ภาษาแอสเซมบลี้ของ Hack กำหนดนีโมนิคในรูปนิพจน์คำนวณดังตาราง

![c-instr-compute-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-compute-field.png)

**หมายเหตุ:** สังเกตว่าตารางมีการกำหนดแพทเทิร์นของบิตไว้ให้สอดคล้องกับขาควบคุมของ ALU ที่เราได้พัฒนาไปก่อนหน้านี้

ฟิลด์ `ddd` เรียกว่า destination field ใช้กำหนดที่หมายเพื่อเก็บผลลัพธ์จากการคำนวณ เลือกที่หมายได้สามแห่งตามค่าของบิตทั้งสาม ได้แก่ รีจีสเตอร์ `A` `D` และ `M` ตามลำดับ โดย `M` หมายถึงตำแหน่งหน่วยความจำที่รีจีสเตอร์ ​`A` ชี้อยู่ ที่หมายทั้งสามนี้สามารถเลือกคละกันได้ หรือเลือกที่จะไม่เก็บผลลัพธ์เลยได้เช่นกัน ดังตารางด้านล่าง

![c-instr-dest-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-dest-field.png)

สุดท้ายคือฟิลด์ `jjj` ใช้ระบุเงื่อนไขการกระโดดตามตารางด้านล่าง ซึ่งหากตรงตามเงื่อนไขจะมีการกระโดดไปยังตำแหน่งหน่วยความจำคำสั่งที่ชี้ด้วยรีจีสเตอร์ `A` เสมอ ดังนั้นก่อนการกระโดดจึงมักต้องใช้ A-instruction เพื่อกำหนดที่หมายเอาไว้ก่อนล่วงหน้า

![c-instr-jump-field.png](https://ecourse.cpe.ku.ac.th/courses/comsys/pics/c-instr-jump-field.png)

### คำสั่งภาษาแอสเซมบลี้สำหรับ C-instruction

รูปแบบคำสั่งภาษาแอสเซมบลี้ของ C-instruction แบบทั่วไปที่สุดคือ

    <dest>=<comp>;<jump>

โดย `dest` `comp` และ `jump` ต้องเขียนเป็นนีโมนิคตามที่ระบุใน 3 ตารางข้างต้นเท่านั้น

หากไม่ต้องการใช้งานฟิลด์นั้น ๆ (ในตารางระบุนีโมนิคเป็น null) ให้ละเอาไว้ ซึ่งมี 2 กรณีคือ

1. ถ้าไม่ต้องการกระโดด ให้เขียนในรูป

       <dest>=<comp>

2. ถ้าไม่ต้องการเก็บผลลัพธ์ของการคำนวณ ให้เขียนในรูป

       <comp>;<jump>

ส่วนฟิลด์คำนวณนั้นต้องมีการระบุเอาไว้เสมอ แม้จะไม่ต้องการคำนวณใด ๆ ก็ตาม เช่นหากต้องการกระโดดโดยไม่มีเงื่อนไข สามารถเขียนคำสั่งในรูป `0;JMP` ได้

### ตัวอย่างคำสั่งที่มีแต่การคำนวณ

พิจารณาตัวอย่างคำสั่งดังนี้  เราเริ่มจากแสดงตัวอย่างการคำนวณทั่วไป

    A=D+1
    M=D+M
    MD=D|M

ทดลองแปลด้วยแอสเซมเบลอร์

In [ ]:
cinst_demo1 = '''
    A=D+1
    M=D+M
    MD=D|M
    A=A+1
'''

insts = assemble(cinst_demo1)
for c, i in zip(cinst_demo1.strip().split("\n"), insts):
    c = c.strip()
    print(f'{c:10}{i:016b}')

แม้ว่าคำสั่งคำนวณจะมีส่วนที่ดูคล้ายคำสั่งในภาษาโปรแกรมระดับสูง เราไม่สามารถสั่งคำสั่งแบบด้านล่างได้

    M=M+D   // ต้องเขียน M=D+M เท่านั้น
    MA=D    // ต้องเขียน AM=D

ทั้งนี้เนื่องจาก `<dest>` และ `<comp>` มีรูปแบบเฉพาะที่แปลงมาจากนีโมนิคที่นิยามให้กับของโค้ดภาษาเครื่อง ไม่ได้เกิดจากการตีความนิพจน์ทางคณิตศาสตร์
 (ยกเว้นจะแก้แอสเซมเบลอร์ให้ฉลาดเกินกว่าที่จำเป็น)

ถ้าพยายามแปลจะพบความผิดพลาดดังนี้

In [ ]:
assemble('M=D+M')

In [ ]:
assemble('AM=D')

### ตัวอย่างคำสั่งที่มีการกระโดด

ด้านล่างแสดงโค้ดที่เปรียบเทียบ `A` กับ `D` และกระโดดไปตำแหน่ง `DEST` ถ้า `A` มากกว่า `D` สังเกตว่าเราต้องลบค่า `A` กับ `D` แล้วเก็บผลลัพธ์ไว้ใน `D` ก่อน เพราะต้องนำเอารีจีสเตอร์ `A` มาโหลดตำแหน่ง `DEST` สำหรับการกระโดด จากนั้นจึงค่อยตรวจสอบค่า `D` และกระโดดตามเงื่อนไขที่ระบุ

        D=A-D
        @DEST
        D;JGT
        ...
    (DEST)
        ...

ส่วนด้านล่างเป็นตัวอย่างการเขียนการกระโดดแบบบังคับ `0;JMP`  ซึ่งเราจะพบรูปแบบนี้ตอนท้ายโปรแกรมต่าง ๆ เพื่อจบโปรแกรม (ทำให้โปรแกรมวนลูปอยู่ที่สองบรรทัดนี้)

    (DONE)
        @DONE
        0;JMP

**ข้อสังเกต** เห็นได้ว่าการอ้างตำแหน่งใน*หน่วยความจำเก็บข้อมูล (RAM)* ด้วยสัญลักษณ์ `M` ไม่ว่าจะอ่านหรือเขียนจะอาศัยค่าจากรีจิสเตอร์ `A` ในการอ้างอิงเสมอ ส่วนการกระโดดนั้นใช้ค่าในรีจีสเตอร์ `A` ระบุตำแหน่งใน*หน่วยความจำเก็บคำสั่ง (ROM)* ซึ่งเป็นหน่วยความจำคนละส่วน ดังนั้นโปรแกรมที่เขียนอย่างถูกต้องจึงไม่ควรมีคำสั่งที่มีทั้ง `M` และการกระโดดปรากฏรวมอยู่ในคำสั่งเดียวกัน

(อย่าลืมดูรายละเอียดอื่น ๆ ในหนังสือเพิ่มเติม)

### ทดลองการกระโดดแบบมีเงื่อนไข

โปรแกรมด้านล่างจะนำค่าใน `R0` ลบด้วย `R1` (หน่วยความจำตำแหน่งที่  0 และ 1 ตามลำดับ) และจะกระโดดไปทำงานที่ `YES` หรือไปที่ `END` ตามเงื่อนไขการกระโดด

ให้ทดลองเปลี่ยนค่าใน `R0` และ `R1` โดยกดที่ช่องใน RAM จะมีให้ใส่ค่าได้ ทดลองเปลี่ยนคำสั่งกระโดดต่าง ๆ และทดลองสั่งให้ทำงาน

**หมายเหตุ:** ให้สังเกตรหัสภาษาเครื่องของคำสั่ง `@END` และ `@YES` ด้วย  ให้ลองคิดล่วงหน้าว่าเมื่อเราจะต้องพัฒนาแอสเซมเบลอร์ด้วยตนเองควรทำอย่างไร (จะต้องทำในสัปดาห์ถัด ๆ ไป)

In [ ]:
#R0/R1 = R3 เศษ R4
jump_demo = '''
    @R0
    D=M
    @R2
    M=D
(Flag)
    @R1
    D=M
    @R2
    M=M-D
    D=M
    @R3
    M=M+1
    @Flag
    D;JGT

    @R2
    D=M
    @Plus
    D;JLT

(Back)

    @R1
    D=M
    @R2
    D=D+M
    @R4
    M=D

(End)
    @End
    0;JMP

(Plus)
    @R3
    M=M-1
    @Back
    0;JMP




'''

listing(jump_demo)
simulate(assemble_and_simulate(jump_demo), memory_display_slots=15)

   0: @R0
   1: D=M
   2: @R2
   3: M=D
      (Flag)
   4: @R1
   5: D=M
   6: @R2
   7: M=M-D
   8: D=M
   9: @R3
  10: M=M+1
  11: @Flag
  12: D;JGT
  13: @R2
  14: D=M
  15: @Plus
  16: D;JLT
      (Back)
  17: @R1
  18: D=M
  19: @R2
  20: D=D+M
  21: @R4
  22: M=D
      (End)
  23: @End
  24: 0;JMP
      (Plus)
  25: @R3
  26: M=M-1
  27: @Back
  28: 0;JMP


TypeError: object of type 'NoneType' has no len()

## ทดลองโปรแกรมบวกเลขตั้งแต่ 1 - N

ด้านล่างเป็นโค้ดภาษาแอสเซมบลี้สำหรับคำนวนผลบวก 1 ถึง `N` โดยเก็บค่า `N` เอาไว้ใน `R0`

**ข้อสังเกต:**
เราใช้ชื่อ `i` และ `sum` ซึ่งแอสเซมเบลอร์จะตีความเป็นตัวแปรและหาตำแหน่งให้เราโดยอัตโนมัติ โดยเริ่มตั้งแต่ตำแหน่ง 16, 17, ... ไปเรื่อย ๆ (ขอให้คิดเอาไว้ล่วงหน้าว่าเมื่อเราต้องเขียนแอสเซมเบลอร์เองจะต้องทำอย่างไร)

**คำถามชวนคิด:** ตัวแปร `i` และ `sum` ถูกกำหนดให้อยู่ที่หน่วยความจำตำแหน่งใด

In [ ]:
sum_1_to_N = """
// Adds 1+...+N, where N is stored in R0
	@i
	M=1
	@sum
	M=0
(LOOP)
	@i
	D=M	    // D=i
	@R0
    A=M     // A=N
	D=D-A	// D=i-N
	@END
	D;JGT	// If (i-N)>0 goto END
	@i
	D=M
	@sum
	M=D+M
	@i
	M=M+1
	@LOOP
	0;JMP
(END)
	@END
	0;JMP
"""

ระบุค่า `R0` โดยคลิก RAM ตำแหน่งที่ 0 แล้วป้อนค่าสำหรับ `N` ลงไป จากนั้นกด Start

In [ ]:
simulate(assemble(sum_1_to_N))

## ทดลองคีย์บอร์ดกับจอแสดงผล

โค้ดต่อไปนี้เขียนให้ทำงานซ้ำ ๆ กันเพื่ออ่านค่าจากสถานะการกดคีย์บอร์ด (จำลอง)
ที่อ่านได้จากหน่วยความจำข้อมูล ณ เลขที่อยู่ 0x6000 ซึ่งสามารถใช้ชื่อพิเศษ `KBD`
ในการอ้างถึงได้ จากนั้นส่งค่าที่ได้ไปยังเลขที่อยู่ของจอแสดงผล ซึ่งเริ่มต้นที่ 0x4000 ซึ่งอ้างถึงได้ด้วยชื่อพิเศษ `SCREEN`
เพื่อให้เห็นค่าของคีย์ได้ชัดเจนขึ้น ค่าคีย์จะถูกวางติดกัน 4 แถวบนหน้าจอ
ซึ่งการอ้างเลขที่อยู่ของเวิร์ดแรกสุดในแถวที่ $i$ ของหน้าจอสามารถคำนวณได้จากสูตร
`0x4000 + 32*i` เนื่องจากจอหนึ่งแถวมีจำนวน 512 จุด
ในหนึ่งแถวจึงเทียบเท่ากับข้อมูล 512/16 = 32 เวิร์ด


In [ ]:
kbd_scr_test = """
(START)
    @KBD
    D=M
    @SCREEN
    M=D
    @16416
    M=D
    @16448
    M=D
    @16480
    M=D
    @START
    0;JMP
"""

ทดลองรัน กดปุ่มที่คีย์บอร์ดจำลองและสังเกตการเปลี่ยนแปลงบนจอแสดงผลจำลอง

In [ ]:
simulate(assemble_and_simulate(kbd_scr_test))

# แบบฝึกหัด

## 1. `Add.asm` - โปรแกรมบวกเลข

เขียนโปรแกรมภาษาแอสเซมบลี้ของสถาปัตยกรรม Hack ที่หาผลบวกของค่าที่อยู่ใน `R0` และ `R1` แล้วเก็บผลลัพธ์ไว้ใน `R2`

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
Add_asm = '''
  @R0
  D=M
  @R1
  D=D+M
  @R2
  M=D
(END)
    @END
    0;JMP
'''

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
simulate(assemble(Add_asm))

### Test case

In [ ]:
from compbuilder.n2t.cpu_sim import PureHackCPU

class TestAsmBase(unittest.TestCase):
    def setUp(self):
        self.cpu = PureHackCPU()

    def assemble_and_run(self, code, max_steps=1000000):
        self.cpu.reset()
        instrs = assemble(code)
        self.cpu.load_instructions(instrs)
        self.cpu.run(max_steps, output_traces=False, until_pc=len(instrs)-1)

In [ ]:
class TestAsmAdd(TestAsmBase):
    def run_add(self, num0, num1):
        self.cpu.ram[0] = num0 % 65536
        self.cpu.ram[1] = num1 % 65536
        self.assemble_and_run(Add_asm)
        self.assertEqual(self.cpu.ram[2], (num0+num1) % 65536)

    def test_add1(self):
        self.run_add(5, 20)

    def test_add2(self):
        self.run_add(5, -2)

    def test_add3(self):
        self.run_add(-30000, 20)

    def test_add4(self):
        self.run_add(-30000, -2)

run_test(TestAsmAdd)

## 2. `Mult.asm` - โปรแกรมหาผลคูณ

เขียนโปรแกรมภาษาแอสเซมบลี้ของสถาปัตยกรรม Hack ที่หาผลคูณของค่าที่อยู่ใน `R0` และ `R1` แล้วเก็บผลลัพธ์ไว้ใน `R2`

**หมายเหตุ:** ถ้าโปรแกรมของคุณยาว สามารถแบ่งโค้ดเป็นหลายส่วนแล้วค่อยมาต่อกันด้วย `+` ก่อนเรียก `assemble` ได้

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
Mult_asm = '''
  @R1
  D=M
  @R3
  M=D

(LOOP)

  @R3
  D=M
  @END
  D;JEQ
  @R3
  M=D-1
  D=M
  @R0
  D=M
  @R2
  M=D+M
  @LOOP
  0;JMP

(END)

    @END
    0;JMP
'''

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
simulate(assemble_and_simulate(Mult_asm))

### Test case

In [ ]:
class TestAsmMult(TestAsmBase):
    def run_mult(self, num0, num1):
        self.cpu.ram[0] = num0 % 65536
        self.cpu.ram[1] = num1 % 65536
        self.assemble_and_run(Mult_asm)
        self.assertEqual(self.cpu.ram[2], (num0*num1) % 65536)

    def test_mult1(self):
        self.run_mult(5, 20)

    def test_mult2(self):
        self.run_mult(5, -2)

    def test_mult3(self):
        self.run_mult(-30000, 20)

    def test_mult4(self):
        self.run_mult(-30000, -2)

run_test(TestAsmMult)

## 3. `Fill.asm` - ระบายหน้าจอ

เขียนโปรแกรมที่อ่านค่าสถานะการกดคีย์บอร์ด

* ถ้ามีการกดปุ่มใด ๆ ค้างไว้ ให้ระบายหน้าจอเป็นสีดำ (บิตของหน้าจอเป็น 1 หมด)
* ถ้าไม่มีการกดปุ่มใด ๆ เลย ให้ระบายหน้าจอเป็นสีขาว (บิตของหน้าจอเป็น 0 หมด)
* การระบายสีดำหรือขาวสามารถเริ่มระบายที่จุดใดของจอก็ได้ แต่ขอให้ระบายให้เต็มทั้งจอตราบเท่าที่มีการกดปุ่ม (หรือปล่อยปุ่ม) ค้างไว้นานพอ

**หมายเหตุ:**
* ถ้าโปรแกรมของคุณยาว สามารถแบ่งโค้ดเป็นหลายส่วนแล้วค่อยมาต่อกันด้วย `+` ก่อนเรียก `assemble` ได้
* โปรแกรมนี้ไม่มีตัวทดสอบอัตโนมัติ ขอให้ทดสอบด้วยตนเองให้แน่ใจว่าทำงานถูกต้อง หากไม่แน่ใจให้เรียกอาจารย์หรือผู้ช่วยสอนตรวจสอบ

**คำใบ้:** สามารถใช้อัลกอริทึมด้านล่างมาแปลงเป็นโค้ดได้

````
while True:
    i = 0
    until i >= 8192:  # screen resolution is 512x256 pixels = (512x256/16) = 8192 words
        if M[KBD] != 0:  # a key is pressed
            M[SCREEN+i] = -1  # blackening 16 pixels
        else:
            M[SCREEN+i] = 0   # whitening 16 pixels
        i = i+1
````

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
Fill_asm = '''
  @0
  D=A
  @i
  M=D

(LOOP)

  @i
  D=M
  @8192
  D=D-A
  @RESET_I
  D;JGE
  @KBD
  D=M
  @BLACK
  D;JNE

(WHITE)

  @i
  D=M
  @SCREEN
  A=D+A
  M=0
  @CONT
  0;JMP

(BLACK)

  @i
  D=M
  @SCREEN
  A=D+A
  M=-1

(CONT)

  @i
  M=M+1
  @LOOP
  0;JMP

(RESET_I)

  @0
  D=A
  @i
  M=D
  @LOOP
  0;JMP

'''

In [ ]:
# พื้นที่ทดสอบ (ปรับเปลี่ยนตามต้องการ)
simulate(assemble_and_simulate(Fill_asm), memory_display_slots=40)

งานส่วนนี้ไม่มีตัวทดสอบอัตโนมัติ

In [ ]:
# DO NOT ERASE THIS CELL - to be graded
Fill_asm = '''
  @0
  D=A
  @i
  M=D
  @KBD
  D=M
  @BLACK
  D;JNE

(WHITE)

    @i
    D=M
    @8192
    D=D-A
    @End
    D;JEQ
    @i
    D=M
    @SCREEN
    A=D+A
    M=0
    @CONTW
    0;JMP

(BLACK)

    @i
    D=M
    @8192
    D=D-A
    @End
    D;JEQ
    @i
    D=M
    @SCREEN
    A=D+A
    M=-1

(CONT)

    @i
    M=M+1
    @BLACK
    0;JMP

(CONTW)

    @i
    M=M+1
    @WHITE
    0;JMP

(End)

    @End
    0;JMP

'''

In [ ]:
simulate(assemble_and_simulate(Fill_asm), memory_display_slots=40)

TypeError: object of type 'NoneType' has no len()